In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

In [6]:
# gpd.read_file('../211-data/SPRE_2021_211cra_join_with_tenure.geo.json')

In [8]:
df_orgs = pd.read_csv('../joined-data/simplified_matches.csv')

df_uw = pd.read_excel('../cra-data/cre-uw-checking 2.xlsx', sheet_name="UW Orgs")
df_uw = df_uw.rename(columns={'CHARITABLE REGISTRATION NUMBER (BN)': 'CRA_BN_ID'})

print(f'Total UW orgs: {len(df_uw)}')

df_uw = df_uw[df_uw['CRA_BN_ID'].notna()]

print(f'Total UW orgs (with CRA BN ID): {len(df_uw)}')

num_uw_matches = len(set(df_uw['CRA_BN_ID']).intersection(set(df_orgs['CRA_BN_ID'])))
print(f'No. UW orgs with 211/CRA data: {num_uw_matches}')

Total UW orgs: 278
Total UW orgs (with CRA BN ID): 261
No. UW orgs with 211/CRA data: 171


In [9]:
df_uw

,Funded Agency - CRA Name,CRA_BN_ID,Category,Sub Category,City,Reason,Region
2,Council of Agencies Serving South Asians,135875599RR0001,NaN,NaN,NaN,BN Not in CRA (in 2021),NaN
7,Human Endeavour Incorporation,857332704RR0001,NaN,NaN,NaN,BN Not in CRA (in 2021),NaN
8,Refugee Career Jumpstart Project,787710490RR0001,NaN,NaN,NaN,BN Not in CRA (in 2021),NaN
9,Brampton 9235 Outreach Corp,132503731RR0001,NaN,NaN,NaN,BN Not in CRA (in 2021),NaN
19,The Reading Partnership,727408932RR0001,NaN,NaN,NaN,BN Not in CRA (in 2021),NaN
...,...,...,...,...,...,...,...
273,Restoration and Empowerment For Social Transit...,811570365RR0001,160.0,19.0,BRAMPTON,Not joined to 211 data,Peel
274,The Canadian Red Cross Society La Soci�t� Cana...,119219814RR0001,100.0,99.0,OTTAWA,Postal Code not included in filter,NaN
275,Dnaagdawenmag Binnoojiiyag Child and Family Se...,830618088RR0001,160.0,19.0,HIAWATHA,Postal Code not included in filter,NaN
276,Routes Connecting Communities Inc.,888551868RR0001,1.0,99.0,SUTTON WEST,Postal Code not included in filter,York


In [10]:
df_orgs

,CRA_BN_ID,CRA_Organization_Name,211_Organization_Name,211_Location_Name,211_Address,211_Postal_Code,X_Coordinate,Y_Coordinate,Match_Method
0,886559590RR0001,CALEDON\DUFFERIN VICTIM SERVICES,CaledonDufferin Victim Services,CaledonDufferin Victim Services - Victim Crisi...,Caledon OPP Station - 15924 Innis Lake Rd,L7C 2Z1,-79.853285,43.876367,Exact Name
1,841536980RR0001,Dr. Jay Children's Grief Centre,Dr Jay Children's Grief Centre,Dr Jay Children's Grief Centre - Dr Jay Childr...,250 Davisville Ave,M4S 1H2,-79.386418,43.700802,Exact Name
2,129904694RR0001,FLEMINGDON HEALTH CENTRE,Flemingdon Health Centre,Flemingdon Health Centre - Prenatal Program,5 Fairview Mall Dr,M2J 2Z1,-79.347820,43.778597,Exact Name
3,129904694RR0001,FLEMINGDON HEALTH CENTRE,Flemingdon Health Centre,Flemingdon Health Centre - Fairview Site,5 Fairview Mall Dr,M2J 2Z1,-79.347820,43.778597,Exact Name
4,129904694RR0001,FLEMINGDON HEALTH CENTRE,Flemingdon Health Centre,Flemingdon Health Centre - Breastfeeding Suppo...,45 Overlea Blvd,M4H 1C3,-79.383200,43.653200,Exact Name
...,...,...,...,...,...,...,...,...,...
2054,850324450RR0001,The Redeemed Christian Church of God-Solid Roc...,Redeemed Christian Church of God,Redeemed Christian Church of God - Food Bank,175 Rexdale Blvd,M9W 1P7,-79.570353,43.710647,General Fuzzy Match
2055,849542832RR0001,Society for Orphaned Armenian Relief - Toronto...,Armenian Relief Society Toronto Roubina Chapter,Armenian Relief Society Toronto Roubina Chapte...,Armenian Community Centre - 45 Hallcrown Place,M2J 4Y4,-79.322158,43.770025,General Fuzzy Match
2056,790514731RR0001,The Redeemed Christian Church of God - Restora...,Redeemed Christian Church of God,Redeemed Christian Church of God - Food Bank,175 Rexdale Blvd,M9W 1P7,-79.570353,43.710647,General Fuzzy Match
2057,119288447RR0001,DAY CARE CONNECTION (TORONTO) INC,Day Care Connection,Day Care Connection - Family Resource Centre,184 Main St,M4E 2W1,-79.300601,43.684288,General Fuzzy Match


In [17]:
df_uw_cra211 = pd.merge(
    left = df_uw,
    right = df_orgs,
    on = 'CRA_BN_ID',
    how = 'left'
)
df_uw_cra211.to_csv('../joined-data/uw_cra211.csv', index=False)